# Procesamiento de lenguaje natural Supermercados JSON

Ahora que tenemos lista la base de datos lista -> empezaremos a crear una solución para poder crear una separación en gramajes cuando exsitan en los titulos de productos.
Además con ayuda de una tecnica de Procesamiento de Lenguaje Natural para extraer palabras clave los productos y poder clasificarlos de mejor manera.

Primero revisemos que tengamos listas estas librerias instaladas ->

In [1]:
#python3 -m spacy download en_core_web_sm

In [2]:
#!python3 -m spacy download es_core_news_sm

El flujo para este desarrollo sera el siguiente:

1. Crearemos el filtrado mediante regex para obtener el gramaje total, validando si son kilogramos o gramos
2. Normalizaremos los datos una vez más
3. Creamos la solución con Procesamiento de Lenguaje Natural para extraer palabras claves

-----------------------------------

### Iniciemos!

De productosChedraui.json y productosSoriana.json obtener los productos que esten en ambos json

In [3]:
import pandas as pd
import json
import re
import pandas as pd
from pymongo import MongoClient
import spacy
from collections import Counter
from spacy.lang.es import STOP_WORDS

In [4]:
pChedraui = []
with open('productosChedraui.json' , encoding="UTF-8") as f:
    file_data = json.load(f)
    for p in file_data:
        nombre = p['nombre']
        precio = p['precio']
        busqueda = p['busqueda']
        url = p['url']
        tienda = p['tienda']
        fecha = p['fecha']
        producto = {
            "precio": precio,
            "busqueda": busqueda,
            "nombre": nombre,
            "url": url,
            "tienda": tienda,
            "fecha": fecha
        }
        pChedraui.append(producto)


In [5]:
import json
pSoriana = []
with open('productosSoriana.json' , encoding="UTF-8") as f:
    file_data = json.load(f)
    for p in file_data:
        nombre = p['nombre']
        precio = p['precio']
        busqueda = p['busqueda']
        url = p['url']
        tienda = p['tienda']
        fecha = p['fecha']
        producto = {
            "precio": precio,
            "busqueda": busqueda,
            "nombre": nombre,
            "url": url,
            "tienda": tienda,
            "fecha": fecha
        }
        pSoriana.append(producto)


In [6]:
pSoriana[2]

{'precio': 309.0,
 'busqueda': 'Comida para gato',
 'nombre': 'Alimento Seco Para Gatos Adultos Purina One  de 2kg',
 'url': 'https://www.soriana.com/alimento-seco-gatos-adultos-purina-one-2kg/11548792.html',
 'tienda': 'Soriana',
 'fecha': '2023-06-12'}

Separar de nombre la marca y el gramaje con regular expresions


In [7]:
for p in pSoriana:
    nombre = p['nombre']
    gramaje = re.findall(r'(\d+)', nombre)
    if len(gramaje) == 0:
        #el producto no tiene gramaje
        p['gramaje'] = "0"
    if len(gramaje) == 1:
         p['gramaje'] = gramaje[0]     
    if len(gramaje) == 2:
        gramaje = gramaje[0] + "." + gramaje[1]
        p['gramaje'] = gramaje

print(pSoriana)


[{'precio': 325.0, 'busqueda': 'Comida para gato', 'nombre': 'Alimento Seco Gatos Adultos Purina One 2kg', 'url': 'https://www.soriana.com/alimento-seco-gatos-adultos-purina-one-2kg/11548792.html', 'tienda': 'Soriana', 'fecha': '2023-06-12', 'gramaje': '2'}, {'precio': 289.0, 'busqueda': 'Comida para gato', 'nombre': 'Alimento Seco Para Gatos Adultos Purina One 2kg', 'url': 'https://www.soriana.com/alimento-seco-gatos-adultos-purina-one-2kg/11548792.html', 'tienda': 'Soriana', 'fecha': '2023-06-12', 'gramaje': '2'}, {'precio': 309.0, 'busqueda': 'Comida para gato', 'nombre': 'Alimento Seco Para Gatos Adultos Purina One  de 2kg', 'url': 'https://www.soriana.com/alimento-seco-gatos-adultos-purina-one-2kg/11548792.html', 'tienda': 'Soriana', 'fecha': '2023-06-12', 'gramaje': '2'}, {'precio': 178.0, 'busqueda': 'Comida para gato', 'nombre': 'FULL LIFE GATOS ADULTOS 1.3KG', 'url': 'https://www.soriana.com/alimento-seco-gatos-adultos-purina-one-2kg/11548792.html', 'tienda': 'Soriana', 'fecha

In [8]:
df = pd.DataFrame(pSoriana)
df

,precio,busqueda,nombre,url,tienda,fecha,gramaje
0,325.0,Comida para gato,Alimento Seco Gatos Adultos Purina One 2kg,https://www.soriana.com/alimento-seco-gatos-ad...,Soriana,2023-06-12,2
1,289.0,Comida para gato,Alimento Seco Para Gatos Adultos Purina One 2kg,https://www.soriana.com/alimento-seco-gatos-ad...,Soriana,2023-06-12,2
2,309.0,Comida para gato,Alimento Seco Para Gatos Adultos Purina One d...,https://www.soriana.com/alimento-seco-gatos-ad...,Soriana,2023-06-12,2
3,178.0,Comida para gato,FULL LIFE GATOS ADULTOS 1.3KG,https://www.soriana.com/alimento-seco-gatos-ad...,Soriana,2023-06-12,1.3
4,275.0,Comida para gato,Alimento Para Gato Whiskas 3 Kg,https://www.soriana.com/alimento-seco-gatos-ad...,Soriana,2023-06-12,3
5,115.0,Comida para gato,Alimento para gato Whiskas esterlizado 1.5kg,https://www.soriana.com/alimento-seco-gatos-ad...,Soriana,2023-06-12,1.5
6,279.0,Comida para gato,Juguete Para Gato Rascadero Casa Zizoo,https://www.soriana.com/alimento-seco-gatos-ad...,Soriana,2023-06-12,0
7,75.0,Comida para gato,Juguete P/Gato Zizoo Raton Natural,https://www.soriana.com/alimento-seco-gatos-ad...,Soriana,2023-06-12,0
8,189.0,Comida para gato,FULL LIFE GATOS GATITOS 1.3 KG,https://www.soriana.com/alimento-seco-gatos-ad...,Soriana,2023-06-12,1.3
9,71.1,Comida para gato,Cepillo Hartz para Gatos,https://www.soriana.com/alimento-seco-gatos-ad...,Soriana,2023-06-12,0


In [9]:
for p in pChedraui:
    nombre = p['nombre']
    gramaje = re.findall(r'(\d+)', nombre)
    if len(gramaje) == 0:
        #el producto no tiene gramaje
        marca = nombre
    if len(gramaje) == 1:
         p['gramaje'] = gramaje[0]     
    if len(gramaje) == 2:
        gramaje = gramaje[0] + "." + gramaje[1]
        p['gramaje'] = gramaje

print(pChedraui)

[{'precio': '100.00', 'busqueda': 'Comida para gato', 'nombre': 'Purina Cat Chow Delimix con Defenseplus Gatos Adultos 1.5kg', 'url': 'https://www.chedraui.com.mx/purina-cat-chow-delimix-con-defenseplus-gatos-adultos-15kg-3258972/p', 'tienda': 'Chedraui', 'fecha': '2023-06-12', 'gramaje': '1.5'}, {'precio': '135.00', 'busqueda': 'Comida para gato', 'nombre': 'Purina Cat Chow Defenseplus Gatitos Sabor Pescado Carne y Leche 1.5kg.', 'url': 'https://www.chedraui.com.mx/purina-cat-chow-defenseplus-gatitos-sabor-pescado-carne-y-leche-15kg-3074427/p', 'tienda': 'Chedraui', 'fecha': '2023-06-12', 'gramaje': '1.5'}, {'precio': '100.00', 'busqueda': 'Comida para gato', 'nombre': 'Purina Cat Chow Defenseplus para Gatos Adultos Sabor Pescado 1.5kg', 'url': 'https://www.chedraui.com.mx/purina-cat-chow-defenseplus-para-gatos-adultos-sabor-pescado-15kg-3110077/p', 'tienda': 'Chedraui', 'fecha': '2023-06-12', 'gramaje': '1.5'}, {'precio': '14.00', 'busqueda': 'Comida para gato', 'nombre': 'Purina Cat

Si el gramaje es menor a 50 entonces esta medido en kilogramos , si es mayor a 50 entonces esta medido en gramos -> 1000 gramos = 1 kilogramo , añadir una columna que sea peso en gramos

In [10]:
for x in pChedraui:
    if float(x['gramaje']) < 50:
        x['gramaje'] = float(x['gramaje']) * 1000
    else:
        x['gramaje'] = float(x['gramaje'])

for x in pSoriana:
    if float(x['gramaje']) < 50:
        x['gramaje'] = float(x['gramaje']) * 1000
    else:
        x['gramaje'] = float(x['gramaje'])

Creamos la columna precio por gramo

In [11]:

for x in pChedraui:
    if float(x['gramaje']) == 0.0:
        x['precioPorGramo'] = 0.0
    else:
        x['precioPorGramo'] = float(x['precio']) / float(x['gramaje'])

for x in pSoriana:
    if float(x['gramaje']) == 0.0:
        print(x)
        x['precioPorGramo'] = 0.0
    else:
        x['precioPorGramo'] = float(x['precio']) / float(x['gramaje'])

{'precio': 279.0, 'busqueda': 'Comida para gato', 'nombre': 'Juguete Para Gato Rascadero Casa Zizoo', 'url': 'https://www.soriana.com/alimento-seco-gatos-adultos-purina-one-2kg/11548792.html', 'tienda': 'Soriana', 'fecha': '2023-06-12', 'gramaje': 0.0}
{'precio': 75.0, 'busqueda': 'Comida para gato', 'nombre': 'Juguete P/Gato Zizoo Raton Natural', 'url': 'https://www.soriana.com/alimento-seco-gatos-adultos-purina-one-2kg/11548792.html', 'tienda': 'Soriana', 'fecha': '2023-06-12', 'gramaje': 0.0}
{'precio': 71.1, 'busqueda': 'Comida para gato', 'nombre': 'Cepillo Hartz para Gatos', 'url': 'https://www.soriana.com/alimento-seco-gatos-adultos-purina-one-2kg/11548792.html', 'tienda': 'Soriana', 'fecha': '2023-06-12', 'gramaje': 0.0}
{'precio': 23.5, 'busqueda': 'Comida para gato', 'nombre': 'Alimento para gato One  cat super nutriente', 'url': 'https://www.soriana.com/alimento-seco-gatos-adultos-purina-one-2kg/11548792.html', 'tienda': 'Soriana', 'fecha': '2023-06-12', 'gramaje': 0.0}


In [12]:
#imprimir pChedraui de manera ordenada
df = pd.DataFrame(pChedraui)
df2 = pd.DataFrame(pSoriana)

Quizas utilizemos almacenarlo en grupos dentro de nuestra base datos!

Unimos los dos dataframes

In [13]:
df = df.reset_index(drop=True)
df2 = df2.reset_index(drop=True)
result = pd.concat([df, df2], ignore_index=True)

In [14]:
# Agrupar los datos por busqueda
grupos = result.groupby('busqueda')

# Obtener el grupo Comida para gato
comida_gato = grupos.get_group('Comida para gato')
print('Comida para gato:')
print(comida_gato)

# Obtener el grupo Mayonesa
mayonesa = grupos.get_group('Mayonesa')
print('Mayonesa:')
print(mayonesa)

# Obtener el grupo Fabuloso
fabuloso = grupos.get_group('Fabuloso')
print('Fabuloso:')
print(fabuloso)

Comida para gato:
    precio          busqueda  \
0   100.00  Comida para gato   
1   135.00  Comida para gato   
2   100.00  Comida para gato   
3    14.00  Comida para gato   
4    14.00  Comida para gato   
5    14.00  Comida para gato   
6    14.00  Comida para gato   
7    14.00  Comida para gato   
24   325.0  Comida para gato   
25   289.0  Comida para gato   
26   309.0  Comida para gato   
27   178.0  Comida para gato   
28   275.0  Comida para gato   
29   115.0  Comida para gato   
30   279.0  Comida para gato   
31    75.0  Comida para gato   
32   189.0  Comida para gato   
33    71.1  Comida para gato   
34   126.0  Comida para gato   
35   115.0  Comida para gato   
36   103.0  Comida para gato   
37    23.5  Comida para gato   
38    30.0  Comida para gato   
39    30.0  Comida para gato   
40    30.0  Comida para gato   

                                               nombre  \
0   Purina Cat Chow Delimix con Defenseplus Gatos ...   
1   Purina Cat Chow Defenseplus Gat

In [15]:
# Conectar con MongoDB
client = MongoClient('localhost', 27017)
db = client['productos']
coleccion = db['grupos']

# Almacenar cada grupo en MongoDB
for nombre_grupo, grupo in grupos:
    grupo_dict = grupo.to_dict(orient='records')
    coleccion.insert_many(grupo_dict)
    print('Grupo', nombre_grupo, 'almacenado en MongoDB.')

Grupo Comida para gato almacenado en MongoDB.
Grupo Fabuloso almacenado en MongoDB.
Grupo Mayonesa almacenado en MongoDB.


## PROCESAMIENTO DE LENGUAJE NATURAL

---------------------------------------

En esta función estamos definiendo la lógica para eliminar la información adicional del producto (en este caso, el peso y la marca) antes de pasar el texto a la función get_keywords. La función convierte el texto a minúsculas, lo divide en palabras y elimina las palabras finales que representan el peso o la marca del producto (si están presentes).

In [16]:
def preprocess(text):
    # Eliminar la información adicional del producto (peso, marca, etc.)
    text = text.lower()
    text = text.split(" ")
    if len(text) > 1:
        if text[-1].isdigit():
            text = " ".join(text[:-2])
        else:
            text = " ".join(text[:-1])
    else:
        text = text[0]
    return text

En esta función estamos definiendo la lógica para extraer las palabras clave del texto. Primero, llamamos a la función preprocess para eliminar la información adicional del producto. Luego, procesamos el texto con el objeto nlp de SpaCy y obtenemos las frases y los tokens relevantes para TextRank (sustantivos, adjetivos, verbos y adverbios). A continuación, calculamos el score de TextRank para cada palabra y ordenamos las palabras por score. Finalmente, seleccionamos las 10 palabras con el score más alto y las filtramos para eliminar las palabras vacías y las frases.

In [17]:
nlp = spacy.load("es_core_news_sm")
def get_keywords(text):
    text = preprocess(text)
    doc = nlp(text)
    keywords = []

    # Obtener las frases del texto
    phrases = [chunk.text for chunk in doc.noun_chunks if not chunk.root.is_stop]

    # Obtener los tokens relevantes para TextRank
    words = [token.text for token in doc if token.is_alpha and not token.is_stop and token.pos_ in ["NOUN", "ADJ", "VERB", "ADV"]]

    # Calcular el score de TextRank para cada palabra
    word_freq = Counter(words)
    scores = {}
    for word in word_freq:
        scores[word] = word_freq[word] / nlp(word).vector_norm

    # Ordenar las palabras por score y añadirlas a las palabras clave
    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_scores[:10]:
        if word not in STOP_WORDS and word not in phrases:
            keywords.append(word)

    return keywords


En este bloque estamos definiendo tres ejemplos de texto que representan nombres de productos de supermercado. Llamamos a la función get_keywords para cada uno de estos

In [18]:
text = "Botana Para Gato 90 Gr Atun Zizoo"
keywords = get_keywords(text)
print(keywords)

text = "Alimento Para Gato Baby Minino 1.3 Kg"
keywords = get_keywords(text)
print(keywords)

text = "Mayonesa Hellmanns Light 800 g"
keywords = get_keywords(text)
print(keywords)

['botana', 'gato']
['baby', 'minino']
['hellmanns', 'light', 'mayonesa']


Realiamos exitosamente el procesamiento de palabras claves. Queda en nosotros elegir las que vamos a utilizar.
Y claro tambien podemos automatizar creando una función que las almacene en un arreglo sin repetir como el que vamos a utilizar llamado __palabras_clave__

In [19]:
import nltk
from nltk.tokenize import word_tokenize

# Definir algunas frases de ejemplo
frases_ejemplo = []
for x in result['nombre']:
    frases_ejemplo.append(x)

In [20]:
# Definir las palabras clave
palabras_clave = ['gato', 'fabuloso', 'purina', 'cat', 'carne', 'alimento', 'mayonesa', 'heinz', 'defenseplus' , 'defensehydro' , 'gatitos', 'c/limon',
                  'hellmann', 'mccormick', 'fabuloso', 'detergente', 'limpiador', 'lavavajillas', 'lavadora', 'lavaplatos', 'lavamanos', 'light',
                  'esterilizados', 'limón', 'antibacterial', 'pasion de frutas', 'lavanda', 'nutrientes', 'mariscos', 'defenseplus', 'Whiskas', 'Minino', 'Carnita']
nombres = result['nombre']

diccionarios = []

# Crear una función que determine si dos listas de tokens tienen palabras clave en común
def tienen_palabras_clave_en_comun(nombres):
    for palabra in result['nombre']:
        palabras_clave_encontradas = []
        tokens = word_tokenize(palabra.lower())
        # print(tokens)
        for token in tokens:

            if token in palabras_clave:
                #print('Frase ->  ',  palabra,
                #      ' Palabra clave encontrada: ', token)
                palabras_clave_encontradas.append(token)

        dic = {"Producto": palabra,
               "Palabras clave encontradas": palabras_clave_encontradas}
        diccionarios.append(dic)


In [21]:
#descargar punkt
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Adolfo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Ejecutamos la función para agregarlas ->

In [22]:
tienen_palabras_clave_en_comun(nombres)

In [23]:
df = pd.DataFrame(diccionarios)
df

,Producto,Palabras clave encontradas
0,Purina Cat Chow Delimix con Defenseplus Gatos ...,"[purina, cat, defenseplus]"
1,Purina Cat Chow Defenseplus Gatitos Sabor Pesc...,"[purina, cat, defenseplus, gatitos, carne]"
2,Purina Cat Chow Defenseplus para Gatos Adultos...,"[purina, cat, defenseplus]"
3,Purina Cat Chow Defensehydro Adultos Pescado A...,"[purina, cat, defensehydro, alimento]"
4,Purina Cat Chow Defensehydro Adultos Carne Ali...,"[purina, cat, defensehydro, carne, alimento]"
5,Purina Cat Chow Defensehydro Adultos Esteriliz...,"[purina, cat, defensehydro, esterilizados]"
6,Purina Cat Chow Defensehydro Adultos Esteriliz...,"[purina, cat, defensehydro, esterilizados]"
7,Purina Cat Chow Defensehydro Adultos Pollo Ali...,"[purina, cat, defensehydro, alimento]"
8,Mayonesa McCormick con limÃ³n 725g,"[mayonesa, mccormick]"
9,Mayonesa McCormick con LimÃ³n 285g,"[mayonesa, mccormick]"


Agregar una columna de palabras clave encontradas a su respectivo producto

In [24]:
result['palabrasClaveEncontradas'] = df['Palabras clave encontradas']
result

,precio,busqueda,nombre,url,tienda,fecha,gramaje,precioPorGramo,palabrasClaveEncontradas
0,100.00,Comida para gato,Purina Cat Chow Delimix con Defenseplus Gatos ...,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-06-12,1500.0,0.066667,"[purina, cat, defenseplus]"
1,135.00,Comida para gato,Purina Cat Chow Defenseplus Gatitos Sabor Pesc...,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-06-12,1500.0,0.090000,"[purina, cat, defenseplus, gatitos, carne]"
2,100.00,Comida para gato,Purina Cat Chow Defenseplus para Gatos Adultos...,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-06-12,1500.0,0.066667,"[purina, cat, defenseplus]"
3,14.00,Comida para gato,Purina Cat Chow Defensehydro Adultos Pescado A...,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-06-12,85.0,0.164706,"[purina, cat, defensehydro, alimento]"
4,14.00,Comida para gato,Purina Cat Chow Defensehydro Adultos Carne Ali...,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-06-12,85.0,0.164706,"[purina, cat, defensehydro, carne, alimento]"
5,14.00,Comida para gato,Purina Cat Chow Defensehydro Adultos Esteriliz...,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-06-12,85.0,0.164706,"[purina, cat, defensehydro, esterilizados]"
6,14.00,Comida para gato,Purina Cat Chow Defensehydro Adultos Esteriliz...,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-06-12,85.0,0.164706,"[purina, cat, defensehydro, esterilizados]"
7,14.00,Comida para gato,Purina Cat Chow Defensehydro Adultos Pollo Ali...,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-06-12,85.0,0.164706,"[purina, cat, defensehydro, alimento]"
8,84.50,Mayonesa,Mayonesa McCormick con limÃ³n 725g,https://www.chedraui.com.mx/mayonesa-mccormick...,Chedraui,2023-06-12,725.0,0.116552,"[mayonesa, mccormick]"
9,39.50,Mayonesa,Mayonesa McCormick con LimÃ³n 285g,https://www.chedraui.com.mx/mayonesa-mccormick...,Chedraui,2023-06-12,285.0,0.138596,"[mayonesa, mccormick]"


Guardar en mongoDB el dataframe result

In [25]:
client = MongoClient('localhost', 27017)
db = client['productos']
coleccion = db['productos']
result_dict = result.to_dict(orient='records')
coleccion.insert_many(result_dict)
print('Grupo', result_dict[0], 'almacenado en MongoDB.')



Grupo {'precio': '100.00', 'busqueda': 'Comida para gato', 'nombre': 'Purina Cat Chow Delimix con Defenseplus Gatos Adultos 1.5kg', 'url': 'https://www.chedraui.com.mx/purina-cat-chow-delimix-con-defenseplus-gatos-adultos-15kg-3258972/p', 'tienda': 'Chedraui', 'fecha': '2023-06-12', 'gramaje': 1500.0, 'precioPorGramo': 0.06666666666666667, 'palabrasClaveEncontradas': ['purina', 'cat', 'defenseplus'], '_id': ObjectId('64876de0d6682ceff9c97b30')} almacenado en MongoDB.


In [26]:
# Leer los datos de MongoDB
cursor = coleccion.find()
# Convertir el cursor a una lista de documentos
documentos = list(cursor)
# Convertir la lista de documentos a un DataFrame de Pandas
df = pd.DataFrame(documentos)

In [27]:
#Seleccionar todos los productos que se llamen Purina Cat Chow Defense Plus para Gatos Adultos Sabor Carne 3kg y agrupar por fecha
df[df['nombre'] == 'Purina Cat Chow Defense Plus para Gatos Adultos Sabor Carne 3kg']

,_id,nombre,busqueda,precio,url,tienda,fecha,gramaje,precioPorGramo,palabrasClaveEncontradas
1,64187e8ca190e8d29890074b,Purina Cat Chow Defense Plus para Gatos Adulto...,Comida para gato,277.00,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-03-20,NaN,NaN,NaN
50,64284e4ac4a2dc63bf488aa0,Purina Cat Chow Defense Plus para Gatos Adulto...,Comida para gato,259.00,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-04-01,NaN,NaN,NaN
107,64286681c4a2dc63bf488ada,Purina Cat Chow Defense Plus para Gatos Adulto...,Comida para gato,259.00,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-04-01,NaN,NaN,NaN
164,6428c54c3fbc3434bd227675,Purina Cat Chow Defense Plus para Gatos Adulto...,Comida para gato,259.00,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-04-01,3000.0,0.086333,NaN
187,642f28f0ec5b8fa3a91e2bd1,Purina Cat Chow Defense Plus para Gatos Adulto...,Comida para gato,259.00,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-04-06,NaN,NaN,NaN
247,642f39a1ec5b8fa3a91e2c0e,Purina Cat Chow Defense Plus para Gatos Adulto...,Comida para gato,259.00,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-04-06,NaN,NaN,NaN
328,643073257d070c556e5298cc,Purina Cat Chow Defense Plus para Gatos Adulto...,Comida para gato,259.00,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-04-07,3000.0,0.086333,"[purina, cat, carne]"
387,6431844abac03a0728da9083,Purina Cat Chow Defense Plus para Gatos Adulto...,Comida para gato,259.00,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-04-08,3000.0,0.086333,"[purina, cat, carne]"
443,6432fc2fe24149dd9de24b99,Purina Cat Chow Defense Plus para Gatos Adulto...,Comida para gato,259.00,https://www.chedraui.com.mx/purina-cat-chow-de...,Chedraui,2023-04-09,3000.0,0.086333,"[purina, cat, carne]"
